In [1]:
#!pip install pandas ## 2.2
#!pip install matplotlib   ## 3.10
#!pip install scikit-learn ## 1.6

#!pip install tensorflow          ## 2.17
#!pip install 'tensorflow[gpu]'   ## 2.17
#!pip install tensorflow-datasets ## 4.9

In [2]:
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

import numpy as np # 1.26
import tensorflow as tf # 2.17
import tensorflow_datasets as tfds # 4.9

np.__version__, tf.__version__, tfds.__version__

ImportError: Traceback (most recent call last):
  File "C:\Users\Angel Temporal\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Error en una rutina de inicialización de biblioteca de vínculos dinámicos (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout
from keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adagrad, Adam, AdamW, Adamax, RMSprop, SGD

In [ ]:
entrada = Input(shape=(4,))
layer1 = Dense(units=3, activation='relu') # oculta
layer2 = Dense(units=4, activation='relu') # oculta
salida = Dense(units=1, activation='relu')

red1 = Sequential( [entrada,layer1,layer2,salida] )

In [ ]:
red2 = Sequential()
red2.add(entrada)
red2.add(layer1)
red2.add(layer2)
red2.add(salida)

In [ ]:
import pandas as pd # 2.2.3
import matplotlib.pyplot as plt # 3.10.0
from sklearn.preprocessing import MinMaxScaler # 1.6.1
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('boston_house_price.csv')
cols_name = df.columns
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
X = df.drop(columns=['MEDV'])
y = df['MEDV']
n_cols = X.shape[1]
X.shape, y.shape

In [ ]:
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=cols_name[:-1])
X.tail()

In [ ]:
X.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=23)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
ntrada = Input(shape=(n_cols,))
salida = Dense(units=1, activation='linear')

### A) instancia de la técnica -> modelo
red3 = Sequential([ntrada,salida])
red3.summary()

In [ ]:
n_epochs = 2_250
batch_size = 10
learn_rate = .002

In [ ]:
optimizador = Adam(learning_rate=learn_rate)
red3.compile(loss="mae", optimizer=optimizador, metrics=["mse"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red3.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

summary.history.keys()

In [ ]:
n_skip = 100
plt.plot(summary.history['mse'][n_skip:], c='b')
plt.plot(summary.history['val_mse'][n_skip:], c='g')

plt.legend(['Train','Valid'], loc='upper right')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show()

In [ ]:
### C) Predicción
y_pred = red3.predict(X_test)[:,0]
### D) Test/Prueba del rendimiento del modelo
RMSE1 = np.sqrt(np.mean((y_test-y_pred)**2))
np.round(RMSE1,3)

In [ ]:
MSE = red3.evaluate(X_test, y_test, verbose=0)[1]
RMSE2 = np.sqrt(MSE)
np.round(RMSE2,3)

#
#
___
#
#

In [ ]:
import pandas as pd # 2.2.3
import matplotlib.pyplot as plt # 3.10.0
from sklearn.preprocessing import MinMaxScaler # 1.6.1
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_iris

In [ ]:
data_raw = load_iris()
X = data_raw['data']
y0= data_raw['target'].reshape(-1,1)

In [ ]:
header = ['SepalLenght', 'SepalWidth','PetalLenght', 'PetalWidth',
      'Setosa', 'Virginica', 'Versicolor']
y = tf.keras.utils.to_categorical(y0, num_classes=3)
df = pd.DataFrame(np.concatenate([X,y],axis=1),columns=header)
df.tail()

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=123)
n_vars = X_train.shape[1]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
ntrada = Input(shape=(n_vars,))
salida = Dense(units=3, activation='softmax')

### A) instancia de la técnica -> modelo
red4 = Sequential([ntrada,salida])
red4.summary()

In [ ]:
n_epochs = 1100
batch_size = 10
learn_rate = .005

optimizador = Adam(learning_rate=learn_rate)
red4.compile(loss="categorical_crossentropy",
     optimizer=optimizador, metrics=["accuracy"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red4.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

summary.history.keys()

In [ ]:
n_skip = 10
plt.plot(summary.history['accuracy'][n_skip:], c='b')
plt.plot(summary.history['val_accuracy'][n_skip:], c='g')

plt.legend(['Train','Valid'], loc='upper right')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
Acc = red4.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {np.round(Acc,3)}')

#
___
#

In [ ]:
ntrada = Input(shape=(n_vars,))
layer1 = Dense(units=6, activation='leaky_relu')
salida = Dense(units=3, activation='softmax')

### A) instancia de la técnica -> modelo
red5 = Sequential([ntrada,layer1,salida])
red5.summary()

In [ ]:
optimizador = Adam(learning_rate=learn_rate)
red5.compile(loss="categorical_crossentropy",
     optimizer=optimizador, metrics=["accuracy"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red5.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

In [ ]:
n_skip = 10
plt.plot(summary.history['accuracy'][n_skip:], c='b')
plt.plot(summary.history['val_accuracy'][n_skip:], c='g')

plt.legend(['Train','Valid'], loc='upper right')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
Acc = red5.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {np.round(Acc,3)}')

#
#
___
#
#

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt # 3.10.0

from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
img0 = X_train[0,:,:]
print(img0.shape)
plt.imshow(img0, cmap='Greys')
plt.show()

In [ ]:
fimg = img0.flatten()
print( fimg.shape )
df = pd.DataFrame(fimg)
df.describe()

In [ ]:
X_train = X_train/255.
X_test  = X_test /255.
n_train_size = X_train.shape[0]
n_train_size, y_test

In [ ]:
y = np.concatenate([y_train,y_test], axis=0)
y = tf.keras.utils.to_categorical(y,10)
y_train = y[:n_train_size,:]
y_test  = y[n_train_size:,:]
y_test.shape, y_test

In [ ]:
drop_prob = 0.5

### A) instancia de la técnica -> modelo
red7 = Sequential()
# Input layer.
red7.add(Input(shape=(28,28,1)))
# 1st convolution + pooling.
red7.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation="relu"))
red7.add(MaxPooling2D(pool_size=2))
# 2nd convolution + pooling.
red7.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation="relu"))
red7.add(MaxPooling2D(pool_size=2))
#Flattened fully connected layer.
red7.add(Flatten())
red7.add(Dense(units=1024, activation="relu"))
# Apply dropout.
red7.add(Dropout(rate=drop_prob))
# Output layer.
red7.add(Dense(units=10, activation="softmax"))

red7.summary()

In [ ]:
n_epochs = 12
batch_size = 200
learn_rate = .001

optimizador = Adam(learning_rate=learn_rate)
red7.compile(loss="categorical_crossentropy",
     optimizer=optimizador, metrics=["accuracy"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red7.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

summary.history.keys()

In [ ]:
n_skip = 0
plt.plot(summary.history['accuracy'][n_skip:], c='b')
plt.plot(summary.history['val_accuracy'][n_skip:], c='g')

plt.legend(['Train','Valid'], loc='upper right')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
Acc = red7.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {np.round(Acc,3)}')

#
___
#

In [ ]:
import matplotlib.pyplot as plt # 3.10.0
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
img0 = X_train[123,:,:]
print(img0.shape)
plt.imshow(img0)
plt.show()

In [ ]:
X_train = X_train/255.
X_test  = X_test /255.
n_train_size = X_train.shape[0]

X_train = X_train.reshape(-1,32,32,3) # 3 canales, c/kernel * matriz de 3*3
X_test  = X_test .reshape(-1,32,32,3)

n_train_size, X_test.shape, y_test

In [ ]:
y = np.concatenate([y_train,y_test], axis=0)
y = tf.keras.utils.to_categorical(y,10)
y_train = y[:n_train_size,:]
y_test  = y[n_train_size:,:]
y_test.shape, y_test

In [ ]:
drop_prob = 0.7

### A) instancia de la técnica -> modelo
red8 = Sequential()
# Input layer.
red8.add(Input(shape=img0.shape)) # (32,32,3)
# 1st convolution + pooling.
red8.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation="relu"))
red8.add(MaxPooling2D(pool_size=2))
# 2nd convolution + pooling.
red8.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation="relu"))
red8.add(MaxPooling2D(pool_size=2))
#Flattened fully connected layer.
red8.add(Flatten())
red8.add(Dense(units=1024, activation="relu"))
# Apply dropout.
red8.add(Dropout(rate=drop_prob))
# Output layer.
red8.add(Dense(units=10, activation="softmax"))

red8.summary()

In [ ]:
n_epochs = 20
batch_size = 20
learn_rate = .0001

optimizador = Adam(learning_rate=learn_rate)
red8.compile(loss="categorical_crossentropy",
     optimizer=optimizador, metrics=["accuracy"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red8.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

summary.history.keys()

In [ ]:
n_skip = 0
plt.plot(summary.history['accuracy'][n_skip:], c='b')
plt.plot(summary.history['val_accuracy'][n_skip:], c='g')

plt.legend(['Train','Valid'], loc='upper right')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
Acc = red8.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {np.round(Acc,3)}')

In [ ]:
from keras.layers import SimpleRNN

In [ ]:
n_input = 1
n_neuron = 100
n_output = 1

red9 = Sequential()
red9.add( SimpleRNN() )

optimizador = Adam(learning_rate=learn_rate)
red9.compile(loss="categorical_crossentropy",
     optimizer=optimizador, metrics=["accuracy"])

### B) Ajustar Parámetros del Modelo a la Data
with tf.device('/GPU:0'):
    summary = red9.fit(X_train, y_train, batch_size=batch_size,
        epochs=n_epochs, validation_split=0.2, verbose=0)

summary.history.keys()